# March Madness Rating Computation

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from data_format.rating_systems.stephenson import *
from data_format.rating_systems.glicko2 import *
from data_format.rating_systems.elo_v2 import *
import copy


In [2]:
DATA_PATH = 'data/'

tourney_results = pd.concat([
    pd.read_csv(DATA_PATH + "mens/MNCAATourneyDetailedResults.csv"),
    pd.read_csv(DATA_PATH + "womens/WNCAATourneyDetailedResults.csv"),
], ignore_index=True)

seeds = pd.concat([
    pd.read_csv(DATA_PATH + "mens/MNCAATourneySeeds.csv"),
    pd.read_csv(DATA_PATH + "womens/WNCAATourneySeeds.csv"),
], ignore_index=True)

regular_results = pd.concat([
    pd.read_csv(DATA_PATH + "mens/MRegularSeasonDetailedResults.csv"),
    pd.read_csv(DATA_PATH + "womens/WRegularSeasonDetailedResults.csv"),
], ignore_index=True)

# regular_results.groupby('Season').mean()
# tourney_results.groupby('Season').mean()
# all(regular_results.columns == tourney_results.columns)


In [3]:
def prepare_data(df):
    dfswap = df[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT', 
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]

    dfswap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
    dfswap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
    df.columns.values[6] = 'location'
    dfswap.columns.values[6] = 'location'    
      
    df.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(df.columns)]
    dfswap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(dfswap.columns)]

    output = pd.concat([df, dfswap]).reset_index(drop=True)
    output.loc[output.location=='N','location'] = '0'
    output.loc[output.location=='H','location'] = '1'
    output.loc[output.location=='A','location'] = '-1'
    output.location = output.location.astype(int)
    
    output['PointDiff'] = output['T1_Score'] - output['T2_Score']

    output['win'] = np.where(output['T1_Score']>output['T2_Score'], 1, 0)

    output['T1_TeamID'] = output['T1_TeamID'].astype(str)
    output['T2_TeamID'] = output['T2_TeamID'].astype(str)
    
    return output

In [4]:
tourney_data = prepare_data(tourney_results)
regular_data = prepare_data(regular_results)

In [5]:
boxscore_cols = ['T1_Score', 'T2_Score', 
        'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_FTM', 'T1_FTA', 'T1_OR', 'T1_DR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_Blk', 'T1_PF', 
        'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_FTM', 'T2_FTA', 'T2_OR', 'T2_DR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk', 'T2_PF', 
        'PointDiff']

boxscore_cols = [
        'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_OR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_PF', 
        'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_OR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk',  
        'PointDiff']

funcs = [np.mean]

In [6]:
season_statistics = regular_data.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs)
season_statistics = regular_data.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs).reset_index()
season_statistics.columns = [''.join(col).strip() for col in season_statistics.columns.values]

/var/folders/ml/c9m4gk2d5j1gjzt21jgb5d3r0000gn/T/ipykernel_12386/1527646356.py:1: FutureWarning: The provided callable <function mean at 0x10621be20> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  season_statistics = regular_data.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs)
/var/folders/ml/c9m4gk2d5j1gjzt21jgb5d3r0000gn/T/ipykernel_12386/1527646356.py:1: FutureWarning: The provided callable <function mean at 0x10621be20> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  season_statistics = regular_data.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs)
/var/folders/ml/c9m4gk2d5j1gjzt21jgb5d3r0000gn/T/ipykernel_12386/1527646356.py:2: FutureWarning: The provided callable <function mean at 0x10621be20> is currently using

In [7]:
season_statistics_T1 = season_statistics.copy()
season_statistics_T2 = season_statistics.copy()

season_statistics_T1.columns = ["T1_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T1.columns)]
season_statistics_T2.columns = ["T2_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T2.columns)]
season_statistics_T1.columns.values[0] = "Season"
season_statistics_T2.columns.values[0] = "Season"

In [8]:
tourney_data = tourney_data[['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID' ,'T2_Score']]
tourney_data = pd.merge(tourney_data, season_statistics_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, season_statistics_T2, on = ['Season', 'T2_TeamID'], how = 'left')

In [9]:
last14days_stats_T1 = regular_data.loc[regular_data.DayNum>118].reset_index(drop=True)
last14days_stats_T1['win'] = np.where(last14days_stats_T1['PointDiff']>0,1,0)
last14days_stats_T1 = last14days_stats_T1.groupby(['Season','T1_TeamID'])['win'].mean().reset_index(name='T1_win_ratio_14d')

last14days_stats_T2 = regular_data.loc[regular_data.DayNum>118].reset_index(drop=True)
last14days_stats_T2['win'] = np.where(last14days_stats_T2['PointDiff']<0,1,0)
last14days_stats_T2 = last14days_stats_T2.groupby(['Season','T2_TeamID'])['win'].mean().reset_index(name='T2_win_ratio_14d')

In [10]:
tourney_data = pd.merge(tourney_data, last14days_stats_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, last14days_stats_T2, on = ['Season', 'T2_TeamID'], how = 'left')

In [11]:
regular_season_effects = regular_data[['Season','T1_TeamID','T2_TeamID','PointDiff']].copy()
regular_season_effects['T1_TeamID'] = regular_season_effects['T1_TeamID'].astype(str)
regular_season_effects['T2_TeamID'] = regular_season_effects['T2_TeamID'].astype(str)
regular_season_effects['win'] = np.where(regular_season_effects['PointDiff']>0,1,0)
march_madness = pd.merge(seeds[['Season','TeamID']],seeds[['Season','TeamID']],on='Season')
march_madness.columns = ['Season', 'T1_TeamID', 'T2_TeamID']
march_madness.T1_TeamID = march_madness.T1_TeamID.astype(str)
march_madness.T2_TeamID = march_madness.T2_TeamID.astype(str)
regular_season_effects = pd.merge(regular_season_effects, march_madness, on = ['Season','T1_TeamID','T2_TeamID'])
regular_season_effects.shape

(20268, 5)

In [12]:
def team_quality(season):
    year_regular_results = regular_results[(regular_results['Season'] == season)]
    # year_tourney_results = tourney_results[(tourney_results['Season'] == season)]

    combined_teams = pd.concat([year_regular_results['T1_TeamID'], year_regular_results['T2_TeamID']]) 
    all_teams = combined_teams.drop_duplicates().tolist()

    team_data = {
        'TeamID': all_teams,   
        'steph_rating': [Stephenson() for _ in all_teams],
    }

    df_team = pd.DataFrame(team_data)

    for index, row in year_regular_results.iterrows():
        team_one = df_team.loc[df_team['TeamID'] == row['T1_TeamID'], 'steph_rating'].values[0]
        team_two = df_team.loc[df_team['TeamID'] == row['T2_TeamID'], 'steph_rating'].values[0]

        #Replacing updateVal since this is not by date but by week 10^2 = 100 (cval)
        team_one.sigma = team_one.sigma + 100
        team_two.sigma = team_two.sigma + 100

        # So we don't evaluate on outcome
        home_clone = copy.deepcopy(team_one)

        point_ratio = float(row['T1_Score']) / (row['T1_Score'] + row['T2_Score'])
        # point_ratio = row['win']

        match row['location']:
            case 'H':
                team_one.newVarRating(team_two, point_ratio, 1) #pone is 1 for home team, -1 away, 0 unknown
                team_two.newVarRating(home_clone, (1 - point_ratio), -1) 
            case 'A':
                team_one.newVarRating(team_two, point_ratio, -1)
                team_two.newVarRating(home_clone, (1 - point_ratio), 1) 
            case _:
                team_one.newVarRating(team_two, point_ratio, 0)
                team_two.newVarRating(home_clone, (1 - point_ratio), 0) 

        df_team.loc[df_team['TeamID'] == row['T1_TeamID'], 'steph_rating'] = team_one
        df_team.loc[df_team['TeamID'] == row['T2_TeamID'], 'steph_rating'] = team_two

    df_team = df_team.assign(quality=df_team['steph_rating'].apply(lambda x: x.getRating()))
    df_team = df_team.sort_values(by="quality", ascending=False)

    # Select only the required columns
    new_team_df = df_team[['TeamID', 'quality']].reset_index(drop=True)
    new_team_df['Season'] = season
    new_team_df['TeamID'] = new_team_df['TeamID'].astype(int)
    return new_team_df

In [13]:
def team_quality_glicko2(season):
    year_regular_results = regular_results[(regular_results['Season'] == season)]
    # year_tourney_results = tourney_results[(tourney_results['Season'] == season)]

    combined_teams = pd.concat([year_regular_results['T1_TeamID'], year_regular_results['T2_TeamID']]) 
    all_teams = combined_teams.drop_duplicates().tolist()

    team_data = {
        'TeamID': all_teams,   
        'steph_rating': [Rating() for _ in all_teams],
    }

    df_team = pd.DataFrame(team_data)

    for index, row in year_regular_results.iterrows():
        team_one = df_team.loc[df_team['TeamID'] == row['T1_TeamID'], 'steph_rating'].values[0]
        team_two = df_team.loc[df_team['TeamID'] == row['T2_TeamID'], 'steph_rating'].values[0]

        # So we don't evaluate on outcome
        home_clone = copy.deepcopy(team_one)

        point_ratio = float(row['T1_Score']) / (row['T1_Score'] + row['T2_Score'])
        # point_ratio = row['win']

        team_one.update_player(team_two.getRating(), team_two.getRd(), point_ratio)
        team_two.update_player(home_clone.getRating(), home_clone.getRd(), (1 - point_ratio))
 
        df_team.loc[df_team['TeamID'] == row['T1_TeamID'], 'steph_rating'] = team_one
        df_team.loc[df_team['TeamID'] == row['T2_TeamID'], 'steph_rating'] = team_two

    df_team = df_team.assign(quality=df_team['steph_rating'].apply(lambda x: x.getRating()))
    df_team = df_team.sort_values(by="quality", ascending=False)

    # Select only the required columns
    new_team_df = df_team[['TeamID', 'quality']].reset_index(drop=True)
    new_team_df['Season'] = season
    new_team_df['TeamID'] = new_team_df['TeamID'].astype(int)
    return new_team_df

In [14]:
# def team_quality_elo(season):
#     year_regular_results = regular_results[(regular_results['Season'] == season)]

#     combined_teams = pd.concat([year_regular_results['T1_TeamID'], year_regular_results['T2_TeamID']]) 
#     all_teams = combined_teams.drop_duplicates().tolist()

#     team_data = {
#         'TeamID': all_teams,   
#         'quality': [E_START_RATING] * len(all_teams)
#     }

#     df_team = pd.DataFrame(team_data)

#     for index, row in year_regular_results.iterrows():
#         team_one = df_team.loc[df_team['TeamID'] == row['T1_TeamID'], 'quality'].values[0]
#         team_two = df_team.loc[df_team['TeamID'] == row['T2_TeamID'], 'quality'].values[0]

#         # point_ratio = float(row['T1_Score']) / (row['T1_Score'] + row['T2_Score'])
#         point_ratio = row['win']

#         new_team_one, new_team_two = base_competing_elo(team_one, team_two, point_ratio)
 
#         df_team.loc[df_team['TeamID'] == row['T1_TeamID'], 'quality'] = new_team_one
#         df_team.loc[df_team['TeamID'] == row['T2_TeamID'], 'quality'] = new_team_two

#     df_team = df_team.sort_values(by="quality", ascending=False)

#     # Select only the required columns
#     new_team_df = df_team[['TeamID', 'quality']].reset_index(drop=True)
#     new_team_df['Season'] = season
#     new_team_df['TeamID'] = new_team_df['TeamID'].astype(int)
#     return new_team_df

In [15]:
glm_quality = pd.concat([team_quality(2010),
                         team_quality(2011),
                         team_quality(2012),
                         team_quality(2013),
                         team_quality(2014),
                         team_quality(2015),
                         team_quality(2016),
                         team_quality(2017),
                         team_quality(2018),
                         team_quality(2019),
                         ##team_quality(2020),
                         team_quality(2021),
                         team_quality(2022),
                         team_quality(2023),
                         team_quality(2024)
                         ]).reset_index(drop=True)

In [16]:
glm_quality_T1 = glm_quality.copy()
glm_quality_T2 = glm_quality.copy()
glm_quality_T1.columns = ['T1_TeamID','T1_quality','Season']
glm_quality_T2.columns = ['T2_TeamID','T2_quality','Season']

glm_quality_T1['T1_TeamID'] = glm_quality_T1['T1_TeamID'].astype(str)
glm_quality_T2['T2_TeamID'] = glm_quality_T2['T2_TeamID'].astype(str)

# tourney_data = tourney_data.loc[tourney_data['DayNum'] >= 136].reset_index(drop=True)

tourney_data['T1_TeamID'] = tourney_data['T1_TeamID'].astype(str)
tourney_data['T2_TeamID'] = tourney_data['T2_TeamID'].astype(str)


In [17]:
tourney_data = pd.merge(tourney_data, glm_quality_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, glm_quality_T2, on = ['Season', 'T2_TeamID'], how = 'left')

In [18]:
seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))

In [19]:
seeds_T1 = seeds[['Season','TeamID','seed']].copy()
seeds_T2 = seeds[['Season','TeamID','seed']].copy()
seeds_T1.columns = ['Season','T1_TeamID','T1_seed']
seeds_T2.columns = ['Season','T2_TeamID','T2_seed']

seeds_T1['T1_TeamID'] = seeds_T1['T1_TeamID'].astype(str)
seeds_T2['T2_TeamID'] = seeds_T2['T2_TeamID'].astype(str)

In [20]:
tourney_data = pd.merge(tourney_data, seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')

In [21]:
tourney_data["Seed_diff"] = tourney_data["T1_seed"] - tourney_data["T2_seed"]

# March Madness teams - team strength overview